### Support Vector Machine
##### By MMA

In [1]:
import numpy as np

### A) Form primal SVM as a QP problem

<img src="equ/Capture11.PNG">

<img src="equ/Capture12.PNG">

** yi must be multiplied to <Xi,1>. A and b is zero!

### B) Solve SVM primal form using cvxopt.solvers.qp

We concatenate w , b and $\xi$ to a vector named Z

In [2]:
from cvxopt import matrix, solvers

In [ ]:
def generate_linear_separable_data():
    # generate training data in the 2-d case
    mean1 = np.array([0, 2])
    mean2 = np.array([2, 0])
    cov = np.array([[0.8, 0.6], [0.6, 0.8]])
    X1 = np.random.multivariate_normal(mean1, cov, 100)
    y1 = np.ones(len(X1))
    X2 = np.random.multivariate_normal(mean2, cov, 100)
    y2 = np.ones(len(X2)) * -1
    return X1, y1, X2, y2


In [4]:
X1, y1, X2, y2 = generate_linear_separable_data()
X_train = np.concatenate((X1[:80], X2[:80]))
y_train = np.concatenate((y1[:80], y2[:80]))
X_test = np.concatenate((X1[80:], X2[80:]))
y_test = np.concatenate((y1[80:], y2[80:]))

In [6]:
C = 0.5
N = 160
d = 2
z_dim = N + d + 1

In [7]:

P = matrix(0, (z_dim, z_dim), 'd')
P[0, 0] = P[1, 1] = 1

In [8]:
q = matrix(0, (z_dim, 1), 'd')
for i in range(3, z_dim):
    q[i] = C

In [9]:
G = matrix(0, (2 * N, N + 2 + 1), 'd')
for i in range(N):
    G[i, 2 + 1 + i] = -1
    G[N + i, 2 + 1 + i] = -1
    G[i, 2] = -1 * y_train[i]
for i in range(N):
    for j in range(2):
        G[i, j] = X_train[i, j] * -1 * y_train[i]


In [11]:
h = matrix(0, (2 * N, 1), 'd')
for i in range(N):
    h[i] = -1


In [12]:
sol = solvers.qp(P, q, G, h)
Z_opt = sol['x']


     pcost       dcost       gap    pres   dres
 0: -3.5063e+01  1.2519e+02  9e+02  3e+00  5e+01
 1:  6.1138e+01 -5.2518e+01  1e+02  3e-01  5e+00
 2:  9.0098e+00 -1.1603e+00  1e+01  1e-02  2e-01
 3:  2.9478e+00  9.3321e-01  2e+00  2e-03  4e-02
 4:  2.4094e+00  1.2647e+00  1e+00  8e-04  2e-02
 5:  2.0480e+00  1.4137e+00  7e-01  3e-04  6e-03
 6:  1.7454e+00  1.6113e+00  1e-01  1e-05  2e-04
 7:  1.6655e+00  1.6623e+00  3e-03  3e-07  5e-06
 8:  1.6636e+00  1.6635e+00  5e-05  4e-09  7e-08
 9:  1.6636e+00  1.6636e+00  5e-07  4e-11  7e-10
Optimal solution found.


In [13]:
w = Z_opt[:d]
b = Z_opt[d]
y_predict = []
for i in range(40):
    y_predict.append(np.sign(np.matmul(np.transpose(w), X_test[i]) + b))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

### C ) Why strong duality is satisfied?

The problem is a QP form thus is a convex problem. On the other hand Slatter's conditions is satisfed because for a data classified correctly but is inside margin, inequality constraint is inactive.

### D) Obtain KKT conditions for this problem.

<img src="equ/Capture.PNG">

### E) Derive dual form of SVM

Get derivative from Lagrangian obtained above :

<img src="equ/Capture5.PNG">

By substituting w and C to Lagrangian we obtain dual function g hence dual problem.

### F) Obtain optimal w,b based on a

<img src="equ/Capture3.PNG">

Each of the samples that has a > 0 is on the margin , thus we solve for b using any of support vectors :

<img src="equ/Capture4.PNG">

### G) Prove that gram matrice for RBF kernel is P.S.D.

Gram matrice is symmetric thus we could prove that all of eigenvalues is non-negative. It is sufficient to investigate 2*2 matrices.

<img src="equ/Capture7.PNG">

### F ) Form dual SVM problem as a QP problem

<img src="equ/Capture10.PNG">

### I ) Implement dual form of SVM using gaussian kernel

In [ ]:
def generate_non_linear_separable_data():
    mean1 = [-1, 2]
    mean2 = [1, -1]
    mean3 = [4, -4]
    mean4 = [-4, 4]
    cov = [[1.0,0.8], [0.8, 1.0]]
    X1 = np.random.multivariate_normal(mean1, cov, 50)
    X1 = np.vstack((X1, np.random.multivariate_normal(mean3, cov, 50)))
    y1 = np.ones(len(X1))
    X2 = np.random.multivariate_normal(mean2, cov, 50)
    X2 = np.vstack((X2, np.random.multivariate_normal(mean4, cov, 50)))
    y2 = np.ones(len(X2)) * -1
    return X1, y1, X2, y2

In [15]:
X1, y1, X2, y2 = generate_non_linear_separable_data()
X_train = np.concatenate((X1[:80], X2[:80]))
y_train = np.concatenate((y1[:80], y2[:80]))
X_test = np.concatenate((X1[80:], X2[80:]))
y_test = np.concatenate((y1[80:], y2[80:]))

In [16]:
diag_y = matrix(0, (n,n), 'd')

for i in range(n):
  diag_y[i, i] = y_train[i]

In [28]:
gamma = 500

import math

In [29]:
Gram = matrix(0, (n,n), 'd')
for i in range(n):
  for j in range(n):
    Gram[i, j] = math.exp(-gamma * (np.linalg.norm (X_train[i]-X_train[j])**2))

P = matrix(np.matmul(np.matmul(diag_y, Gram), diag_y.T))

P

<160x160 matrix, tc='d'>

In [30]:
q = matrix(-1, (n,1), 'd')

A = matrix(0, (1,n), 'd')
for i in range(n):
  A[0,i] = y_train[i]

b = matrix(0, (1,1), 'd')

G = matrix(-1, (2,n), 'd')
for i in range(n):
  G[1, i] = 1
print(G)

h = matrix(0, (2,1), 'd')
h[1, 0] = C

In [31]:
sol=solvers.qp(P, q, G, h, A, b)
alpha = (sol['x'])

     pcost       dcost       gap    pres   dres
 0: -7.4586e-01 -9.9940e-01  4e+00  2e+00  6e-16
 1: -4.0653e-01 -8.1799e-01  4e-01  4e-16  8e-16
 2: -4.7993e-01 -5.0240e-01  2e-02  8e-17  7e-16
 3: -4.9902e-01 -4.9925e-01  2e-04  2e-16  6e-16
 4: -4.9921e-01 -4.9921e-01  2e-06  7e-16  4e-16
 5: -4.9921e-01 -4.9921e-01  2e-08  5e-16  1e-16
Optimal solution found.


In [32]:
w = np.zeros((1,2))
for i in range(n):
  w = w + alpha[i] * y_train[i] * X_train[i]

w

array([[ 0.42528238, -0.34247832]])

In [33]:
sv = X_train[np.argmax(alpha)]
sv_label = y_train[np.argmax(alpha)]
temp = 0
for i in range(n):
    temp = temp + alpha[i] * y_train[i] * math.exp(-gamma * (np.linalg.norm(X_train[i] - sv) ** 2))
b = sv_label - temp

In [34]:
y_predict_list = []
for i in range(40):
    temp = 0
    for j in range(n):
        temp = temp + alpha[j] * y_train[j] * math.exp(-gamma * (np.linalg.norm(X_train[j] - X_test[i]) ** 2))
    if b + temp >= 0:
        y_predict = 1
    else:
        y_predict = -1
    y_predict_list.append(y_predict)

In [36]:
index = 0
for i in range(40):
    if y_predict_list[i] == y_test[i]:
        index += 1
print(index/40)

0.5


In [ ]:
## TODO : correct diag_y -1 to transpose of diag_y , change 1 to -1 in G for primal SVM